In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from config import weather_api_key
import requests
from citipy import citipy

### Step 3
Create a new set of 2,000 random latitudes and longitudes.

In [5]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=10500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=10500)
lat_lngs = zip(lats, lngs)

In [6]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

### Step 4
Get the nearest city using the `citipy` module.

In [7]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

2296

### Step 5 and 6
Perform an API call with the OpenWeatherMap.

Retrieve the following information from the API call:

Latitude and longitude,
Maximum temperature,
Percent humidity,
Percent cloudiness,
Wind speed,
Weather description (for example, clouds, fog, light rain, clear sky)

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=imperial&appid=" + weather_api_key

In [ ]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    
# Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_country = city_weather["sys"]["country"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_weather_description = city_weather['weather'][0]['description']
        
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description": city_weather_description})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [11]:
len(city_data)

2133

### Step 7
Add the data to a new DataFrame

In [14]:
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Rikitea,PF,-23.1203,-134.9692,78.98,69,5,7.52,clear sky
1,Punta Arenas,CL,-53.1500,-70.9167,44.71,70,0,8.05,clear sky
2,Hilo,US,19.7297,-155.0900,73.80,90,0,0.00,clear sky
3,Kahramanmaras,TR,38.0000,37.0000,21.83,100,97,4.03,overcast clouds
4,Ketchikan,US,55.3422,-131.6461,45.07,70,40,9.22,scattered clouds
...,...,...,...,...,...,...,...,...,...
2128,Bayan,CN,46.0833,127.4000,6.94,82,100,7.76,overcast clouds
2129,Gwembe,ZM,-16.4975,27.6069,69.28,82,96,1.79,light rain
2130,Arcata,US,40.8665,-124.0828,43.66,80,0,3.44,mist
2131,Sorso,IT,40.8000,8.5772,50.49,92,100,8.43,overcast clouds


### Step 8
Export the DataFrame as a CSV file, and save it as `WeatherPy_Database.csv` in the `Weather_Database` folder.

In [15]:
# Create the output file (CSV).
output_data_file = "Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")